In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import json
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [34]:
response = requests.get("https://www.lesoir.be/18/sections/le-direct")
response.text

'<!doctype html>\n<html lang="fr" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article#">\n<head>\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n  <meta name="viewport" content="width=device-width, initial-scale=1.0">\n  <title>Le direct - Le Soir</title>\n      <script>dataLayer=[{"typeUser":"Anonymous",\n"typePage":"Destination",\n"nodetype":"",\n"path":"18/sections/le-direct",\n"nodeid":"",\n"termid":"",\n"nodepublicationdate":"",\n"nodepublicationhour":"",\n"nodeeditorialupdatedate":"",\n"nodeeditorialupdatehour":"",\n"dpipagetitle":"Actualités",\n"nodetitle":"",\n"userID":"",\n"userstatut":"anonyme",\n"gender":"",\n"emailhash":"",\n"maindestname":"",\n"Statut":"Blocked",\n"role":"anonymous user",\n"breadcrumb":"soirplus/ledirect",\n"RestrictedAccess":"",\n"breadcrumb2":["soirplus",\n"ledirect"],"sensitive":"",\n"member_status_v2":"anonyme",\n"articlelength

In [35]:
soup = bs(response.content, "html.parser")
articles = soup.find_all("article")
print(len(articles))
articles[0]


20


<article class="r-viewmode--article">
<time class="r-direct--time" datetime="2023-08-01T15:44:28">15h44</time>
<i-article><a class="r-article--img" href="http://soirmag.lesoir.be/528807/article/2023-08-01/reconnu-non-coupable-dagressions-sexuelles-kevin-spacey-devrait-faire-son-retour"> <!-- scald=30342657:ena_manchette {"entity":{"type":"node","bundle":"package","id":"528807","vid":"528807"},"class":""} -->
<figure aria-label="Belgaimage-71151535" class="r-ratio--16-9" role="img">
<img alt="Belgaimage-71151535" class="r-img lazy" data-sizes="auto" data-src="https://leseng.rosselcdn.net/sites/default/files/dpistyles_v2/ls_16_9_606w/2023/08/01/node_528807/30342657/public/2023/08/01/B9734828650Z.1_20230801154428_000%2BGHHN7LPJS.1-0.jpg?itok=Eoypckgx1690899309" data-srcset="https://leseng.rosselcdn.net/sites/default/files/dpistyles_v2/ls_16_9_1124w/2023/08/01/node_528807/30342657/public/2023/08/01/B9734828650Z.1_20230801154428_000%2BGHHN7LPJS.1-0.jpg?itok=GzXyFMdE1690899309 1124w, https:/